In [1]:
import torch
from torch import nn
from models.ResLayer import ResLayer
from models.ResNet import ResNet
import torchvision

In [2]:
x = torch.rand(1,3,32,32)
config = {
        "n_classes": 10,
        "batch_size": 1024,
        "lr": 3e-4,
        "gradient_clip_val": 0.5,
        "dropout": 0.5,
        "num_epochs": 50,
        "layer0_in": 3,
        "layer1_in": 16,
        "layer2_in": 32,
        "layer3_in": 64,
        "classifier_in": 256,
        "layer0_kernel": 3,
        "layer0_stride": 1,
        "layer0_padding": 1,
        "res_kernel": 3
    }

In [3]:
layer0 = nn.Sequential(
                nn.Conv2d(in_channels=config['layer0_in'], out_channels=config['layer1_in'], \
                        kernel_size=config['layer0_kernel'], stride=config['layer0_stride'], padding=config['layer0_padding']),
                nn.MaxPool2d(kernel_size=config['layer0_kernel'], stride=config['layer0_stride'], padding=config['layer0_padding']),
                nn.BatchNorm2d(config['layer1_in']),
                nn.ReLU()
                )

In [4]:
l0 = layer0(x)
l0.shape

torch.Size([1, 16, 32, 32])

In [5]:
layer1 = ResLayer(in_channels=config['layer1_in'], out_channels=config['layer2_in'], config=config)

In [16]:
layer1.resblock1.conv2(layer1.resblock1.conv1(l0)).shape

torch.Size([1, 32, 16, 16])

In [12]:
l1 = layer1(l0)
l1.shape

torch.Size([1, 32, 16, 16])

In [7]:
layer2 = ResLayer(in_channels=config['layer2_in'], out_channels=config['layer3_in'], config=config)

In [8]:
l2 = layer2(l1)
l2.shape

torch.Size([1, 64, 8, 8])

In [10]:
layer3 = ResLayer(in_channels=config['layer3_in'], out_channels=config['classifier_in'], config=config)

In [11]:
l3 = layer3(l2)
l3.shape

torch.Size([1, 256, 4, 4])

In [22]:
pooling = nn.AdaptiveAvgPool2d(1)
pooling(l3).shape

torch.Size([1, 256, 1, 1])

In [6]:
resnet = ResNet(config)
resnet(x).shape

torch.Size([10])

In [60]:
res50 = torchvision.models.resnet50(pretrained=True)
res50.conv1 = torch.nn.Conv2d(
    3, 64, kernel_size=3, stride=1, padding=1, bias=False
)
res50.maxpool = torch.nn.Identity()
res50

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Identity()
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentu

In [61]:
res50(x).shape

torch.Size([1, 1000])

In [55]:
for module in res50.named_modules():
    print(module[0])


conv1
bn1
relu
maxpool
layer1
layer1.0
layer1.0.conv1
layer1.0.bn1
layer1.0.conv2
layer1.0.bn2
layer1.0.conv3
layer1.0.bn3
layer1.0.relu
layer1.0.downsample
layer1.0.downsample.0
layer1.0.downsample.1
layer1.1
layer1.1.conv1
layer1.1.bn1
layer1.1.conv2
layer1.1.bn2
layer1.1.conv3
layer1.1.bn3
layer1.1.relu
layer1.2
layer1.2.conv1
layer1.2.bn1
layer1.2.conv2
layer1.2.bn2
layer1.2.conv3
layer1.2.bn3
layer1.2.relu
layer2
layer2.0
layer2.0.conv1
layer2.0.bn1
layer2.0.conv2
layer2.0.bn2
layer2.0.conv3
layer2.0.bn3
layer2.0.relu
layer2.0.downsample
layer2.0.downsample.0
layer2.0.downsample.1
layer2.1
layer2.1.conv1
layer2.1.bn1
layer2.1.conv2
layer2.1.bn2
layer2.1.conv3
layer2.1.bn3
layer2.1.relu
layer2.2
layer2.2.conv1
layer2.2.bn1
layer2.2.conv2
layer2.2.bn2
layer2.2.conv3
layer2.2.bn3
layer2.2.relu
layer2.3
layer2.3.conv1
layer2.3.bn1
layer2.3.conv2
layer2.3.bn2
layer2.3.conv3
layer2.3.bn3
layer2.3.relu
layer3
layer3.0
layer3.0.conv1
layer3.0.bn1
layer3.0.conv2
layer3.0.bn2
layer3.0.conv

In [26]:
for param in res50.parameters():
    param.requires_grad = False

In [45]:
res50.conv1

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

In [49]:
res50.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=2, padding=1)

In [50]:
res50.conv1(x).shape

torch.Size([1, 64, 16, 16])

In [44]:
res50.conv1(x).shape

torch.Size([1, 64, 16, 16])

In [27]:
for param in res50.parameters():
    if param.requires_grad:
        print(param.names)

In [9]:
i = 0
newres = torch.nn.Sequential(*list(res50.children())[:-2])

In [12]:
newres[-1]

Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(1024, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2d(1024, 2048, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (1): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (1): Bottleneck(
    (conv1): Conv2d(2048, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): 